In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder,LabelEncoder
import pickle

In [2]:
data =pd.read_csv('diabetes.csv')
data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
import pandas as pd
df =pd.read_csv('gender_submission.csv')
df.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [5]:
x =data.drop('Outcome',axis=1)
y=data['Outcome']


#### split the data set into train and test
x_train,x_test,y_train,y_test =train_test_split(x,y,test_size=0.2,random_state=42)


##scale the features
scaler =StandardScaler()
x_train =scaler.fit_transform(x_train)
x_test =scaler.fit_transform(x_test)

In [6]:
with open('scaler1.pkl','wb') as file:
    pickle.dump(scaler,file)

## ANN implementation


In [8]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime




In [9]:
x_train.shape[1]

8

In [10]:
### build the ann model
model =Sequential([
    Dense(32,activation='relu',input_shape=(x_train.shape[1],)),
    Dense(16,activation='relu',input_shape=('relu')),
    Dense(1,activation='sigmoid')
])

d:\gen-ai\anaconda\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [11]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 833 (3.25 KB)

 Trainable params: 833 (3.25 KB)

 Non-trainable params: 0 (0.00 B)

In [12]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [13]:
### set up tensor board
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
lod_dir='dlog/fit/' +datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback =TensorBoard(log_dir=lod_dir,histogram_freq=1)

In [14]:
early_callback= EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [15]:
history =model.fit(
    x_train,y_train,validation_data =(x_test,y_test),
    epochs =50,
    callbacks =[tensorflow_callback,early_callback]
)

Epoch 1/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.4739 - loss: 0.7114 - val_accuracy: 0.6169 - val_loss: 0.6852
Epoch 2/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6954 - loss: 0.6541 - val_accuracy: 0.6883 - val_loss: 0.6446
Epoch 3/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7345 - loss: 0.6136 - val_accuracy: 0.6948 - val_loss: 0.6080
Epoch 4/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7410 - loss: 0.5742 - val_accuracy: 0.7143 - val_loss: 0.5767
Epoch 5/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7557 - loss: 0.5418 - val_accuracy: 0.7403 - val_loss: 0.5528
Epoch 6/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7590 - loss: 0.5149 - val_accuracy: 0.7403 - val_loss: 0.5356
Epoch 7/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7590 - loss: 0.4961 - val_accuracy: 0.7468 - val_loss: 0.5242
Epoch 8/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7638 - loss: 0.4815 - val_accuracy: 0.7403 - val_loss

In [16]:
model.save('dmodel.h5')

In [17]:
%load_ext tensorboard

In [19]:
%tensorboard --logdir dlog/fit

Reusing TensorBoard on port 6006 (pid 18056), started 0:00:08 ago. (Use '!kill 18056' to kill it.)

In [22]:
from tensorflow.keras.models import load_model
import pickle

model = load_model("dmodel.h5")

with open("scaler1.pkl", "rb") as f:
    scaler = pickle.load(f)


In [23]:
# Example new patient
new_patient = [[2, 120, 70, 25, 100, 28.5, 0.5, 33]]
# [Pregnancies, Glucose, BloodPressure, SkinThickness, Insulin, BMI, DiabetesPedigreeFunction, Age]


In [24]:
new_patient_scaled = scaler.transform(new_patient)


d:\gen-ai\anaconda\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [25]:
prediction =model.predict(new_patient_scaled)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step


In [26]:
prediction

array([[0.20502168]], dtype=float32)

In [27]:
prediction_proba =prediction[0][0]

In [28]:
prediction_proba

np.float32(0.20502168)

In [ ]:
if prediction_proba >0.5:
    print("the customer likely to have diabetes")
else:
    print("less likely to have diabetes")

less likely to have diabetes


: 